
## 4. <a id='4_cell'></a>Create Group using Excel, Pandas and the PyFHIR models....

This uses Pandas to easily upload patient data from an Excel (or CSV) file based
on some tricks I learned to create instancea fo resources a lot more complex than patient such as conformance
resources like CapabilityStatement and StructureDefinitition.

### Load and excel file using Pandas

I've made a sample excel file `my_patients.xlsx` containing data for several patients.
Note that you will need to install the *pandas* module.

- See this set of tutorials to learn more about Pandas:

   [![pandas_tut](pandas.png)](https://www.youtube.com/watch?v=ZyhVh-qRZPA&list=PL-osiE80TeTsWmV9i9c58mdDCSskIFdDS "Pandas Tutorial")

In [11]:
cd ~/pyFHIR_models # This is to make sure in right directory for Binder

[Errno 2] No such file or directory: '/Users/ehaas/pyFHIR_models # This is to make sure in right directory for Binder'
/Users/ehaas/Documents/Python/MyBinder/pyFHIR_models


In [12]:
from pandas import *
df = read_excel('my_patients.xlsx','Sheet1',na_filter = False, dtype=str)
# treat all empty cells as '' and convert all data including booleans to string  

df

,row,my_id,f_name,l_name,gender,b_date,is_deceased,is_active,phone,address1,city,state,zip
0,1,123,Amy,Brown,F,1964-06-19 00:00:00,False,True,5555555555,100 Main St,Napa,CA,94559
1,2,234,Bert,Black,M,1969-08-09 00:00:00,False,True,5555555555,100 Oak St,Napa,CA,94558
2,3,345,Colleen,Blue,F,2000-09-21 00:00:00,False,False,5555555555,100 Spruce St,Napa,CA,94559
3,4,456,Daniel,Crimson,M,1996-05-23 00:00:00,False,True,5555555555,100 Yahome St,Napa,CA,94558
4,5,567,Elizabeth,Green,F,1941-10-21 00:00:00,False,True,5555555555,100 Trancas Blvd,Napa,CA,94559
5,6,689,Frederick,Gray,M,1964-06-26 00:00:00,False,True,5555555555,100 A St,Napa,CA,94558


### How to Access Dataframe Data Using dot.notation

To do this use the [itertuples method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.itertuples.html).

In [3]:
for i in df.itertuples(index=True):     
    print(f'i.row = {i.row}: i.f_name = {i.f_name}, i.l_name = {i.l_name}, i.gender = {i.gender} , etc...')

i.row = 1: i.f_name = Amy, i.l_name = Brown, i.gender = F , etc...
i.row = 2: i.f_name = Bert, i.l_name = Black, i.gender = M , etc...
i.row = 3: i.f_name = Colleen, i.l_name = Blue, i.gender = F , etc...
i.row = 4: i.f_name = Daniel, i.l_name = Crimson, i.gender = M , etc...
i.row = 5: i.f_name = Elizabeth, i.l_name = Green, i.gender = F , etc...
i.row = 6: i.f_name = Frederick, i.l_name = Gray, i.gender = M , etc...


### Create Group Instance, One member for Each Row...

1. initialize group resource and datatype instances
1. iterate over dataframe rows
1. assign dataframe data to member elements (may need to massage the strings to fit the data-types - see `active` for example)
1. validate and save the resources in the reference server
1. option to create narrative using jinja2 templates (that is a topic for another notebook)

### import all the modules you need

In [4]:
from fhir_model_generator.model import fhirdate, fhirreference, group, patient, identifier, humanname, contactpoint, address
from json import dumps
from requests import get, post
from datetime import datetime, date, timedelta
from IPython.display import display as Display, HTML, Markdown

### Initialize the Group Resource

In [5]:
my_g = group.Group(
        dict(
            type = 'person',
            actual = True,
            name = "Example provider patients for today",
            active = True,
            quantity = df.shape[0],          
        )
) # initialize group instance

print(dumps(my_g.as_json(), indent =4))

{
    "resourceType": "Group",
    "active": true,
    "type": "person",
    "actual": true,
    "name": "Example provider patients for today",
    "quantity": 6
}


### add an identifier and the managingEntity element - i.e. the Organization that created the group.

In [7]:
my_id = identifier.Identifier( # add complex identifier element
    dict(
        value = f'group-{str(datetime.now().isoformat())}',
        system = "http://example.org",
        ))
my_g.identifier = [my_id]

my_me = fhirreference.FHIRReference(
    dict(
        #reference = "http://example.org",
        display = "example managingEntity",
    )
)
my_g.managingEntity = my_me

print(dumps(my_g.as_json(), indent =4))

{
    "resourceType": "Group",
    "identifier": [
        {
            "system": "http://example.org",
            "value": "group-2020-03-21T21:19:48.463955"
        }
    ],
    "active": true,
    "type": "person",
    "actual": true,
    "name": "Example provider patients for today",
    "quantity": 6,
    "managingEntity": {
        "display": "example managingEntity"
    }
}


### Add Group Characteristics

In [8]:
today = str(date.today())
tommorrow = str(date.today() + timedelta(days=1))

today, tommorrow

('2020-03-21', '2020-03-22')

In [9]:
my_gc = group.GroupCharacteristic(
        dict(
            exclude = False,
            code =  {"text":"provider"},
            valueReference =  {
                "reference": "Practitioner/X",
                "display": "Doctor X",
            },
            period = {
                "start": today,
                "end": tommorrow,
                    }
        ),
)

my_g.characteristic = [my_gc]

print(dumps(my_g.as_json(), indent =4))

{
    "resourceType": "Group",
    "identifier": [
        {
            "system": "http://example.org",
            "value": "group-2020-03-21T21:19:48.463955"
        }
    ],
    "active": true,
    "type": "person",
    "actual": true,
    "name": "Example provider patients for today",
    "quantity": 6,
    "managingEntity": {
        "display": "example managingEntity"
    },
    "characteristic": [
        {
            "code": {
                "text": "provider"
            },
            "valueReference": {
                "reference": "Practitioner/X",
                "display": "Doctor X"
            },
            "exclude": false,
            "period": {
                "start": "2020-03-21",
                "end": "2020-03-22"
            }
        }
    ]
}


### Finally Add the Group Member from Excel

- use the business identifier instead of the FHIR Reference.

In [10]:
my_g.member =[]
for i in df.itertuples(index=True):
    my_member = group.GroupMember(
        dict(
            inactive= False,
            entity = dict(
                identifier = dict(
                        value = i.my_id,
                        system = "http://example.org",
                    ),
                display = f'MRN = {i.my_id}',
            ),
        )
    )
    
    my_g.member.append(my_member)

    
my_gj = dumps(my_g.as_json(), indent =4)

print(my_gj)

{
    "resourceType": "Group",
    "identifier": [
        {
            "system": "http://example.org",
            "value": "group-2020-03-20T09:31:57.886132"
        }
    ],
    "active": true,
    "type": "person",
    "actual": true,
    "name": "Example provider patients for today",
    "quantity": 6,
    "managingEntity": {
        "display": "example managingEntity"
    },
    "characteristic": [
        {
            "code": {
                "text": "provider"
            },
            "valueReference": {
                "reference": "Practitioner/X",
                "display": "Doctor X"
            },
            "exclude": false,
            "period": {
                "start": "2020-03-20",
                "end": "2020-03-21"
            }
        }
    ],
    "member": [
        {
            "entity": {
                "identifier": {
                    "system": "http://example.org",
                    "value": "123"
                },
                "display": "M

   
### ======= Validate  =======
    

In [108]:
ref_server ='http://hapi.fhir.org/baseR4'
headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
params = dict(
      )
print('validating ...')
r = post(f'{ref_server}/{my_g.resource_type}/$validate',
         params = params, headers = headers, data = my_gj)
display(HTML(
    '<h1>Validation output</h1>'
    f'<h3>Status Code = {r.status_code}</h3>'
    f'{r.json()["text"]["div"]}'
    ))

validating ...


WARNING,"[Group, Line 1, Col 2]",dom-6: A resource should have narrative for robust management [text.div.exists()]
INFORMATION,"[Group.characteristic[0].code, Line 21, Col 14]","Binding for path Group.characteristic[0].code has no source, so can't be checked"


### ======= Save/Load to ref FHIR Server ===========
  

In [112]:
r = post(f'{ref_server}/{my_g.resource_type}',
          params = params, headers = headers, data = my_gj)
try:
    display(HTML(
        '<h1>Post Response</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
        f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
        '===============================================<br /><br /><br />'
        ))
except KeyError:
    display(HTML(
        '<h1>Post Response</h1>'
        f'<h3>Status Code = {r.status_code}</h3><br />'
        f'<pre>Response Headers: {dumps(dict(r.headers), indent=4)}</pre>'
        #f'<em>Resource Narrative</em>: {r.json()["text"]["div"]}'
        '===============================================<br /><br /><br />'
        ))   

###  Todo:

- Add Narratives
- Fetch ids for patients to replace the identifiers